In [74]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.utils import shuffle

In [75]:
# Load dataset
#vegis_df = pd.read_csv("G:/Meine Ablage/KI_Projekt/Daten/vegis_traindata.csv")
#vegis_df = vegis_df.drop(columns=["Unnamed: 0", "green pixels", "Image"])
#print(vegis_df.head())
#print(vegis_df.shape)
X, y = load_iris(return_X_y=True)
print(X[:5])
print(y)
print("Shape X: ", X.shape)
print("Shape y: ", y.shape)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
Shape X:  (150, 4)
Shape y:  (150,)


In [76]:
df = pd.DataFrame(X)
df['4'] = y
df.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [77]:
df2 = shuffle(df)
df2.head()

,0,1,2,3,4
51,6.4,3.2,4.5,1.5,1
54,6.5,2.8,4.6,1.5,1
22,4.6,3.6,1.0,0.2,0
48,5.3,3.7,1.5,0.2,0
139,6.9,3.1,5.4,2.1,2


In [78]:
iris_train, iris_test = df2.iloc[0:131, :], df2.iloc[131:, :]
print(iris_train.head())
print(iris_test.head())

       0    1    2    3  4
51   6.4  3.2  4.5  1.5  1
54   6.5  2.8  4.6  1.5  1
22   4.6  3.6  1.0  0.2  0
48   5.3  3.7  1.5  0.2  0
139  6.9  3.1  5.4  2.1  2
       0    1    2    3  4
41   4.5  2.3  1.3  0.3  0
127  6.1  3.0  4.9  1.8  2
16   5.4  3.9  1.3  0.4  0
114  5.8  2.8  5.1  2.4  2
53   5.5  2.3  4.0  1.3  1


In [79]:
iris_train_x = np.array(iris_train)
iris_test_x = np.array(iris_test)
print(iris_train_x.shape)
print(iris_test_x.shape)

(131, 5)
(19, 5)


In [69]:
df = pd.DataFrame(X)
df['4'] = y
df.head()
dataset_iris = np.array(df)

In [71]:
np.random.shuffle(dataset_iris)

In [73]:
print(dataset_iris.shape)
y = dataset_iris[0:25,:]
print(y.shape)
print(print(dataset_iris.shape))

(150, 5)
(25, 5)
(150, 5)
None


In [61]:
class DecissionTree:
    def __init__(self, max_depth, min_size):
        self.max_depth = max_depth
        self.min_size = min_size
        self.depth = 0
        self.classes_dataset = None
        self.tree = None

    def fit(self, dataset):
        """
        X: array(n,d)
        y: array(labels)
        """
        #self.classes_dataset = list(set(dataset[:,-1]))
        
        # Nach jedem Split gibt es ein neuen Datensatz der dann übergeben werden muss 
        root = self._get_split(dataset)
        self.depth += 1
        self._split(root)
        self.tree = root
        return 

    def _split(self, node):
        left, right = node['buckets']
        del(node['buckets'])
        #print("Left: ", left)
        #print("Right: ", right)
        # Check if there was no split. The lists are empty
        if not left or not right:
            # Add amount of each class to the node
            node['left'] = node['right'] = self._to_terminal(left + right)
            return
        if self.depth >= self.max_depth:
            node['left'], node['right'] = self._to_terminal(left), self._to_terminal(right)
            return
        if len(left) <= self.min_size:
            node['left'] = self._to_terminal(left)
        else:
            node['left'] = self._get_split(left)
            self.depth += 1
            self._split(node['left'])
        
        if len(right) <= self.min_size:
            node['right'] = self._to_terminal(right)
        else:
            node['right'] = self._get_split(right)
            self.depth += 1
            self._split(node['right'])
        
    def _get_split(self, dataset):
        best_feat, best_thresh, best_gini_loss, best_buckets = 999, 999, 999, None
        #unique_cls_values = list(set(dataset[:,-1]))
        print("DATASET: ,", dataset)
        unique_cls_values = list(set(row[-1] for row in dataset))
        # Do the split for each feature in the dataset (X)
        for idx in range(len(dataset[0])-1): # do not use the label for split
            # assign all instances to a split
            print("Split for ", idx)
            for row in dataset:
                buckets = self._create_split(idx, row[idx], dataset)
            # evaluate the split for the two groups
            gini_loss = self._gini_loss(buckets, unique_cls_values)
            print("New Gini loss: ", gini_loss)

            # check if the split is better than the current best split. If true, keep it
            if gini_loss < best_gini_loss:
                best_gini_loss = gini_loss
                best_feat = idx
                best_thresh = row[idx]
                best_buckets = buckets

                #print(f"best gini: {best_gini_loss}\nbest feat Idx: {best_feat}\nbest thresh: {best_thresh}\nbest buckets: {best_buckets}")
        return {'feature_idx': best_feat, 'thresh': best_thresh, 'buckets': best_buckets}

    def _create_split(self, idx, thresh, dataset):
        # idx = current feature for the split
        # thresh = current threshold for the split. Now check value for the whole dataset. Then grab the next threshold
        left_bucket, right_bucket = list(), list()
        for row in dataset:
            if row[idx] <= thresh:
                left_bucket.append(row)  #append row instead of only the label
            else:
                right_bucket.append(row)

        return left_bucket, right_bucket

    def _gini_loss(self, buckets, cls_values):
        # cls_values: Are the values in the current dataset 
        #print("Class Values i ngini : ", cls_values)
        n_instances = float(sum(len(bucket) for bucket in buckets))
        #print("Number Instances: ", n_instances)

        gini = 0.0
        for bucket in buckets:
            size = float(len(bucket))
            # calculate gini score for left and right bucket
            if size == 0:
                continue
            score = 0.0
            for cls_val in cls_values:
                ratio = [row[-1] for row in bucket].count(cls_val) / size
                #ratio = bucket.count(cls_val) / size
                score += ratio**2

            gini += (size / n_instances) * (1.0 - score)
        return gini
    
    def _to_leaf(self, bucket):
        pred_cls = {cls:0 for cls in self.classes_dataset}
        for c in self.classes_dataset:
            pred_cls[c] = bucket.count(c)
        return pred_cls
    
    def _to_terminal(self, bucket):
        outcomes = [row[-1] for row in bucket]
        return max(set(outcomes), key=outcomes.count)
    
    def predict(self, X_test):
        predictions = list()
        node = self.tree
        for row in X_test:
            pred = self._make_predictions(node, row)
            predictions.append(pred)
        return predictions
    
    def _make_predictions(self, node, row):
        if row[node['feature_idx']] < node['thresh']:
            if isinstance(node['left'], dict):
                return self._make_predictions(node['left'], row)
            else:
                return node['left'] #final prediction
        else:
            if isinstance(node['right'], dict):
                return self._make_predictions(node['right'], row)
            else:
                return node['right'] #final prediction
        




In [81]:
clf_tree = DecissionTree(15, 4)
clf_tree.fit(iris_train_x)

DATASET: , [[6.4 3.2 4.5 1.5 1. ]
 [6.5 2.8 4.6 1.5 1. ]
 [4.6 3.6 1.  0.2 0. ]
 [5.3 3.7 1.5 0.2 0. ]
 [6.9 3.1 5.4 2.1 2. ]
 [5.  3.6 1.4 0.2 0. ]
 [5.6 3.  4.1 1.3 1. ]
 [6.9 3.1 5.1 2.3 2. ]
 [6.7 2.5 5.8 1.8 2. ]
 [5.8 2.7 3.9 1.2 1. ]
 [6.3 3.3 6.  2.5 2. ]
 [6.6 3.  4.4 1.4 1. ]
 [5.1 3.8 1.6 0.2 0. ]
 [5.2 3.4 1.4 0.2 0. ]
 [5.5 2.4 3.8 1.1 1. ]
 [7.1 3.  5.9 2.1 2. ]
 [6.8 3.2 5.9 2.3 2. ]
 [5.  2.  3.5 1.  1. ]
 [5.1 2.5 3.  1.1 1. ]
 [7.2 3.  5.8 1.6 2. ]
 [5.4 3.7 1.5 0.2 0. ]
 [5.  3.3 1.4 0.2 0. ]
 [7.7 2.8 6.7 2.  2. ]
 [6.4 2.8 5.6 2.1 2. ]
 [5.1 3.8 1.9 0.4 0. ]
 [4.7 3.2 1.6 0.2 0. ]
 [4.6 3.2 1.4 0.2 0. ]
 [6.  2.7 5.1 1.6 1. ]
 [5.  3.2 1.2 0.2 0. ]
 [5.7 3.  4.2 1.2 1. ]
 [6.3 3.3 4.7 1.6 1. ]
 [7.2 3.2 6.  1.8 2. ]
 [5.8 2.6 4.  1.2 1. ]
 [5.1 3.5 1.4 0.3 0. ]
 [7.9 3.8 6.4 2.  2. ]
 [7.6 3.  6.6 2.1 2. ]
 [6.2 2.9 4.3 1.3 1. ]
 [5.8 2.7 4.1 1.  1. ]
 [6.3 3.4 5.6 2.4 2. ]
 [5.5 4.2 1.4 0.2 0. ]
 [6.  3.  4.8 1.8 2. ]
 [6.7 3.3 5.7 2.1 2. ]
 [6.7 3.  5.  1.7 1. ]


In [82]:
print(clf_tree.tree)

{'feature_idx': 2, 'thresh': 1.9, 'left': {'feature_idx': 0, 'thresh': 4.8, 'left': {'feature_idx': 0, 'thresh': 4.8, 'left': 0.0, 'right': 0.0}, 'right': {'feature_idx': 0, 'thresh': 4.9, 'left': 0.0, 'right': {'feature_idx': 0, 'thresh': 5.1, 'left': {'feature_idx': 0, 'thresh': 5.1, 'left': 0.0, 'right': 0.0}, 'right': {'feature_idx': 0, 'thresh': 5.8, 'left': 0.0, 'right': 0.0}}}}, 'right': {'feature_idx': 2, 'thresh': 5.2, 'left': {'feature_idx': 3, 'thresh': 2.0, 'left': {'feature_idx': 0, 'thresh': 6.5, 'left': {'feature_idx': 1, 'thresh': 3.0, 'left': {'feature_idx': 0, 'thresh': 6.5, 'left': 1.0, 'right': 1.0}, 'right': {'feature_idx': 1, 'thresh': 3.2, 'left': 1.0, 'right': 1.0}}, 'right': {'feature_idx': 0, 'thresh': 6.7, 'left': 1.0, 'right': 1.0}}, 'right': 2.0}, 'right': {'feature_idx': 0, 'thresh': 6.3, 'left': 2.0, 'right': 2.0}}}


In [83]:
pred = clf_tree.predict(iris_test_x)
pred

[0.0,
 1.0,
 0.0,
 2.0,
 1.0,
 0.0,
 1.0,
 2.0,
 2.0,
 0.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 0.0]

In [84]:
iris_test_x[:,-1]

array([0., 2., 0., 2., 1., 0., 1., 2., 2., 0., 2., 2., 1., 1., 1., 1., 2.,
       2., 0.])

In [41]:
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
dataset_np = np.array(dataset)
dataset_np.shape

In [62]:
t = tree.fit(dataset_np)

DATASET: , [[ 2.77124472  1.78478393  0.        ]
 [ 1.72857131  1.16976141  0.        ]
 [ 3.67831985  2.81281357  0.        ]
 [ 3.96104336  2.61995032  0.        ]
 [ 2.99920892  2.20901421  0.        ]
 [ 7.49754587  3.16295355  1.        ]
 [ 9.00220326  3.33904719  1.        ]
 [ 7.44454233  0.47668338  1.        ]
 [10.12493903  3.23455098  1.        ]
 [ 6.64228735  3.31998376  1.        ]]
Split for  0
New Gini loss:  0.1666666666666666
Split for  1
New Gini loss:  0.4444444444444444
DATASET: , [array([2.77124472, 1.78478393, 0.        ]), array([1.72857131, 1.16976141, 0.        ]), array([3.67831985, 2.81281357, 0.        ]), array([3.96104336, 2.61995032, 0.        ]), array([2.99920892, 2.20901421, 0.        ]), array([6.64228735, 3.31998376, 1.        ])]
Split for  0
New Gini loss:  0.2777777777777777
Split for  1
New Gini loss:  0.2777777777777777
DATASET: , [array([7.49754587, 3.16295355, 1.        ]), array([9.00220326, 3.33904719, 1.        ]), array([7.44454233, 0.4

In [64]:
print(tree.tree)

{'feature_idx': 0, 'thresh': 6.642287351, 'left': {'feature_idx': 0, 'thresh': 6.642287351, 'left': 0.0, 'right': 0.0}, 'right': {'feature_idx': 0, 'thresh': 10.12493903, 'left': 1.0, 'right': 1.0}}


In [65]:
pred = tree.predict(dataset_np)
pred

[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]